# Initial Set Up

In [ ]:
from google.cloud import bigquery
from google.cloud import aiplatform
from vertexai.generative_models import GenerativeModel

In [ ]:
project_id = "your project id here" # @param {"type":"string"}
location = "us-central1" # @param {"type":"string"}

dataset_name = "ecommerce" # @param {"type":"string"}
model_name = "customer_segmentation_model" # @param {"type":"string"}
table_name = "customer_stats" # @param {"type":"string"}

client = bigquery.Client(project=project_id, location=location)
aiplatform.init(project=project_id, location=location)


In [ ]:
%%bigquery
CREATE SCHEMA IF NOT EXISTS ecommerce_bqml OPTIONS(
  location="us")
;

Query is running:   0%|          |

""


# Create control and test data for Contribution Analysis

Contribution analysis models take a single table as input.

For a summable contribution analysis model, the following columns are required: a numerical metric column (total_sales in this example), a boolean column to indicate whether a record is in the test or control set, and one or more categorical columns which form the ‘contributors’.

In [ ]:
%%bigquery
CREATE OR REPLACE TABLE ecommerce_bqml.iowa_liquor_sales_control_and_test AS
(SELECT
  store_name,
  city,
  vendor_name,
  category_name,
  item_description,
  SUM(sale_dollars) AS total_sales,
  FALSE AS is_test
FROM `bigquery-public-data.iowa_liquor_sales.sales`
WHERE EXTRACT(YEAR FROM date) = 2020
GROUP BY store_name, city, vendor_name,
  category_name, item_description, is_test
)
UNION ALL
(SELECT
  store_name,
  city,
  vendor_name,
  category_name,
  item_description,
  SUM(sale_dollars) AS total_sales,
  TRUE AS is_test
FROM `bigquery-public-data.iowa_liquor_sales.sales`
WHERE EXTRACT(YEAR FROM date) = 2021
GROUP BY store_name, city, vendor_name,
  category_name, item_description, is_test
);

Query is running:   0%|          |

""


# Run Contribution Analysis using BigQuery ML

## Create model

To create a contribution analysis model, you can use the `CREATE MODEL` statement.
In this example, we are interested in the total_sales summable metric and in the store_name, city, vendor_name, category_name, and item_description as potential contributor dimensions.
To reduce model creation time and exclude small segments of data, we are adding a minimum support value of `0.05`. The minimum support value of 0.05 guarantees that output segments must make up at least 5% of the total_sales in the underlying test or control data.

In [ ]:
%%bigquery
CREATE OR REPLACE MODEL ecommerce_bqml.iowa_liquor_sales_contribution_analysis_model
  OPTIONS(
    model_type = 'CONTRIBUTION_ANALYSIS',
    contribution_metric =
      'sum(total_sales)',
    dimension_id_cols = ['store_name', 'city',
      'vendor_name', 'category_name', 'item_description'],
    is_test_col = 'is_test',
    min_apriori_support = 0.05
) AS
SELECT * FROM ecommerce_bqml.iowa_liquor_sales_control_and_test;

Query is running:   0%|          |

""


## Get insights from the model
With the model created in the previous step, you can use the new `ML.GET_INSIGHTS` function to retrieve the insights from the sales data.



In [ ]:
%%bigquery contribution_analysis_df_positive
SELECT
  contributors,
  metric_test,
  metric_control,
  difference,
  relative_difference,
  unexpected_difference,
  relative_unexpected_difference,
  apriori_support
 FROM ML.GET_INSIGHTS(
  MODEL ecommerce_bqml.iowa_liquor_sales_contribution_analysis_model)
WHERE relative_difference > 0
ORDER BY unexpected_difference DESC
LIMIT 10;

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
contribution_analysis_df_positive.head(10)

,contributors,metric_test,metric_control,difference,relative_difference,unexpected_difference,relative_unexpected_difference,apriori_support
0,[all],4.280682e+08,3.964730e+08,3.159522e+07,0.079691,3.159522e+07,0.079691,1.000000
1,[vendor_name=SAZERAC COMPANY INC],5.232731e+07,3.886473e+07,1.346257e+07,0.346396,1.149192e+07,0.281421,0.122241
2,[category_name=100% AGAVE TEQUILA],2.391510e+07,1.725217e+07,6.662926e+06,0.386208,5.528662e+06,0.300692,0.055868
3,[city=DES MOINES],4.952132e+07,4.174677e+07,7.774549e+06,0.186231,4.971158e+06,0.111586,0.115686
4,[vendor_name=BACARDI USA INC],2.390247e+07,1.912217e+07,4.780297e+06,0.249987,3.421456e+06,0.167055,0.055838
5,[category_name=STRAIGHT BOURBON WHISKIES],3.449522e+07,3.015587e+07,4.339349e+06,0.143897,2.095597e+06,0.064680,0.080583
6,[vendor_name=DIAGEO AMERICAS],8.468107e+07,7.725926e+07,7.421814e+06,0.096064,1.571127e+06,0.018904,0.197821
7,"[category_name=CANADIAN WHISKIES, vendor_name=...",2.502043e+07,2.210785e+07,2.912571e+06,0.131744,1.218738e+06,0.051204,0.058450
8,[vendor_name=FIFTH GENERATION INC],2.596442e+07,2.337973e+07,2.584685e+06,0.110552,7.667516e+05,0.030429,0.060655
9,"[category_name=AMERICAN VODKAS, vendor_name=FI...",2.596442e+07,2.337973e+07,2.584685e+06,0.110552,7.667516e+05,0.030429,0.060655


# Use LLM to interpret the results

## Set up a function to convert raw results into prompts

In [ ]:
# prompt: Using contribution_analysis_df, which contains result of contribution analysis. Create explanation what are the top 5 factors contributing to the change in sales, is it positive change or negative change and how much impact both in terms of absolute number and percentage change. If the contributor of index 0 is [all], skip index 0.

def analyze_contribution(df, top_n=5):
  """Analyzes the top contributors to sales change.

  Args:
    df: DataFrame containing contribution analysis results.
    top_n: Number of top contributors to analyze.

  Returns:
    A list of strings, each describing a contributor's impact.
  """

  explanations = []
  start_index = 0
  # if df['contributors'][0] == ['all']:
  #   start_index = 1 # skip first row if [all]
  # else:
  #   start_index = 0
  for i in range(start_index, top_n + start_index + 1):
    if i >= len(df):
      break
    row = df.iloc[i]
    contributors = ', '.join(row['contributors'])
    change = row['difference']
    change_unexpected = row['unexpected_difference']
    relative_change = row['relative_difference'] * 100
    relative_change_unexpected = row['relative_unexpected_difference'] * 100

    if change > 0:
      change_direction = 'increased'
    else:
      change_direction = 'decreased'

    explanation = (
        f"Factor {i}: Sales from {contributors} {change_direction} by "
        f"{abs(change):.2f} with {change_unexpected} of the change in this segment considered unexpected when compared with the population as a whole (or change by {relative_change_unexpected:.2f}% more than the total change)."
    )
    explanations.append(explanation)
  return explanations



## Call functions and prepare prompts

In [ ]:
# Get the top 5 explanations
top_explanations = analyze_contribution(contribution_analysis_df_positive, top_n=5)

# Print the explanations
for explanation in top_explanations:
  print(explanation)

Factor 0: Sales from all increased by 31595222.45 with 31595222.449922442 of the change in this segment considered unexpected when compared with the population as a whole (or change by 7.97% more than the total change).
Factor 1: Sales from vendor_name=SAZERAC COMPANY  INC increased by 13462573.12 with 11491923.259674296 of the change in this segment considered unexpected when compared with the population as a whole (or change by 28.14% more than the total change).
Factor 2: Sales from category_name=100% AGAVE TEQUILA increased by 6662926.02 with 5528662.409690235 of the change in this segment considered unexpected when compared with the population as a whole (or change by 30.07% more than the total change).
Factor 3: Sales from city=DES MOINES increased by 7774549.40 with 4971158.268011183 of the change in this segment considered unexpected when compared with the population as a whole (or change by 11.16% more than the total change).
Factor 4: Sales from vendor_name=BACARDI USA INC in

In [ ]:
prompt_1 = f"""
You're a marketing strategist, given the following results from contributor analysis, come up with \
simple explanation as if you are explaining what the analysis does to a CEO, and describe the business impact by covering: \
(1) For 'Factor 0: Sales from all', this is the total overall change between the two period. Explain this first. \
(2) For each of the remaining factors, describe what are the top factors, what is the impact (positive to negative), and the magnitude of the impact \
(3) Describe next possible marketing actions to further improve positive results or to rectify negative results, do it step by step.

The comparison is internal benchmark between the control period and the test period. This is not about comparing with market trends.

Factors:
{top_explanations}

For each factor:
* Contributor:
* Change direction and amount:
* Next marketing step:

Put extra line breaks between each factor, but no extra line breaks between Contributor, Change direction and amount, and Next marketing step of the same factor
"""

print(prompt_1)


You're a marketing strategist, given the following results from contributor analysis, come up with simple explanation as if you are explaining what the analysis does to a CEO, and describe the business impact by covering: (1) For 'Factor 0: Sales from all', this is the total overall change between the two period. Explain this first. (2) For each of the remaining factors, describe what are the top factors, what is the impact (positive to negative), and the magnitude of the impact (3) Describe next possible marketing actions to further improve positive results or to rectify negative results, do it step by step.

The comparison is internal benchmark between the control period and the test period. This is not about comparing with market trends.

Factors:
['Factor 0: Sales from all increased by 31595222.45 with 31595222.449922442 of the change in this segment considered unexpected when compared with the population as a whole (or change by 7.97% more than the total change).', 'Factor 1: Sal

# Call LLM to interpret results

In [ ]:
# Use the Vertex AI API to call the model and generate a marketing campaign using the variable prompt.

model = GenerativeModel("gemini-1.5-pro")

responses = model.generate_content(
    prompt_1,
    generation_config={
        "temperature": 0.5,
        "max_output_tokens": 5000,
        "top_p": 1.0,
        "top_k": 40,
    }
  )

# for response in responses:
#     print(response.text)
print(responses.text)

## Explanation of Contributor Analysis for CEO

This analysis helps us understand which factors contributed the most to the change in sales between our control period and test period. It goes beyond simply saying "sales went up," and pinpoints *what* specifically drove that change.

**Factor 0: Sales from all**

This simply tells us the total sales increase between the two periods.

* **Contributor:** Overall Sales
* **Change direction and amount:** Increased by $31,595,222.45 
* **Next marketing step:**  This serves as our baseline. We'll compare other factors against this to see their relative impact.


**Factor 1: Sales from vendor_name=SAZERAC COMPANY  INC**

* **Contributor:** Sales of products from Sazerac Company Inc. 
* **Change direction and amount:** Increased by $13,462,573.12 (28.14% higher than the average increase)
* **Next marketing step:** This is a significant driver of our sales growth!  We should analyze what's working well with Sazerac products – are there specific 

In [ ]:
## example calling Gemini 1.5 Flash instead

model = GenerativeModel("gemini-1.5-flash")

responses = model.generate_content(
    prompt_1,
    generation_config={
        "temperature": 0.5,
        "max_output_tokens": 5000,
        "top_p": 1.0,
        "top_k": 40,
    }
  )

# for response in responses:
#     print(response.text)
print(responses.text)

## Contributor Analysis: Understanding Sales Changes

This analysis helps us understand the key drivers behind our sales changes between the control and test periods. It looks at how different factors contribute to the overall sales increase or decrease, highlighting areas that performed significantly better or worse than expected.

### Factor 0: Sales from all

* **Contributor:** Overall sales across all segments
* **Change direction and amount:** Sales increased by 31595222.45 (7.97% more than the total change).
* **Business Impact:** This represents the total sales increase observed during the test period. It's important to note that 7.97% of this increase was unexpected, meaning it wasn't solely driven by general market trends.

### Factor 1: Sales from vendor_name=SAZERAC COMPANY INC

* **Contributor:** Sales from the vendor "SAZERAC COMPANY INC"
* **Change direction and amount:** Sales increased by 13462573.12 (28.14% more than the total change).
* **Business Impact:** Sales from

# Negative factors

Follow similar steps but now focusing on factors contributing to decline in sales

In [ ]:
%%bigquery contribution_analysis_df_negative
SELECT
  contributors,
  metric_test,
  metric_control,
  difference,
  relative_difference,
  unexpected_difference,
  relative_unexpected_difference,
  apriori_support
FROM ML.GET_INSIGHTS(MODEL ecommerce_bqml.iowa_liquor_sales_contribution_analysis_model)
WHERE relative_difference < 0
ORDER BY unexpected_difference
LIMIT 5;

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
contribution_analysis_df_negative.head(5)

,contributors,metric_test,metric_control,difference,relative_difference,unexpected_difference,relative_unexpected_difference,apriori_support
0,[vendor_name=HEAVEN HILL BRANDS],27021134.0,27769681.88,-748547.88,-0.026956,-3.184589e+06,-0.10543,0.070042


In [ ]:
# Get the top 5 explanations
top_explanations = analyze_contribution(contribution_analysis_df_negative, top_n=5)

# Print the explanations
for explanation in top_explanations:
  print(explanation)

Factor 0: Sales from vendor_name=HEAVEN HILL BRANDS decreased by 748547.88 with -3184588.6584070027 of the change in this segment considered unexpected when compared with the population as a whole (or change by -10.54% more than the total change).


In [ ]:
prompt_2 = f"""
You're a marketing strategist, given the following results from contributor analysis, come up with \
simple explanation as if you are explaining what the analysis does to a CEO, and describe the business impact by covering: \
(1) For each of the factors, describe what are the top factors, what is the impact (positive to negative), and the magnitude of the impact \
(2) Describe next possible marketing actions to further improve positive results or to rectify negative results, do it step by step.

When explaining, start from factor 1 as the first factor and continue increasing the number by 1. Do not use 0 as a starting number.
The comparison is internal benchmark between the control period and the test period. This is not about comparing with market trends.

Factors:
{top_explanations}

For each factor:
* Contributor:
* Change direction and amount:
* Next marketing step:

Put extra line breaks between each factor, but no extra line breaks between Contributor, Change direction and amount, and Next marketing step of the same factor
"""

print(prompt_1)


You're a marketing strategist, given the following results from contributor analysis, come up with simple explanation as if you are explaining what the analysis does to a CEO, and describe the business impact by covering: (1) For 'Factor 0: Sales from all', this is the total overall change. Explain this first. (2) For each of the remaining factors, describe what are the top factors, what is the impact (positive to negative), and the magnitude of the impact (3) Describe next possible marketing actions to further improve positive results or to rectify negative results, do it step by step.

The comparison is internal benchmark between the control period and the test period. This is not about comparing with market trends.

Factors:
['Factor 0: Sales from all increased by 31595222.45 with 31595222.4499228 of the change in this segment considered unexpected when compared with the population as a whole (or change by 7.97% more than the total change).', 'Factor 1: Sales from vendor_name=SAZER

In [ ]:
model = GenerativeModel("gemini-1.5-pro")

responses = model.generate_content(
    prompt_2,
    generation_config={
        "temperature": 0.5,
        "max_output_tokens": 800,
        "top_p": 1.0,
        "top_k": 40,
    }
  )

# for response in responses:
#     print(response.text)
print(responses.text)


Good morning/afternoon [CEO Name],

I’m here today to walk you through the results of our recent contributor analysis. This analysis helps us understand which factors had the biggest impact on our sales performance, comparing a control period to a recent test period we ran. This allows us to double down on what’s working and course-correct on what isn’t.

Let’s dive into the findings:

***

**Factor 1**

* **Contributor:** Sales from the vendor "HEAVEN HILL BRANDS" 
* **Change direction and amount:** Decreased by $748,547.88. This drop is significantly larger than the overall change in this segment, accounting for 10.54% more of the decrease than expected.
* **Next marketing step:** 
    1. **Investigate:** We need to understand why sales from HEAVEN HILL BRANDS plummeted.  Was there a change in our product mix, pricing, or promotions? Were there external factors like supply chain issues or a competitor's promotion impacting this specific vendor?
    2. **Address the root cause:** Depe

In [ ]:
model = GenerativeModel("gemini-1.5-flash")

responses = model.generate_content(
    prompt_2,
    generation_config={
        "temperature": 0.5,
        "max_output_tokens": 800,
        "top_p": 1.0,
        "top_k": 40,
    }
  )

# for response in responses:
#     print(response.text)
print(responses.text)


## Contributor Analysis Explained

This analysis helps us understand which factors contributed the most to our recent performance changes, comparing the control period (our baseline) to the test period (the recent period we're analyzing).  This helps us identify areas where we're doing well and areas where we need to focus our efforts.

**Factor 1:**

* **Contributor:** Sales from vendor_name=HEAVEN HILL BRANDS
* **Change direction and amount:** Sales decreased by $748,547.88. This is a significant drop, and the analysis shows that -3,184,588.66 of this change was unexpected compared to the overall trend. This means the decline in sales from this vendor was much larger than we would have expected based on the overall market performance. 
* **Next marketing step:**  We need to investigate why sales from HEAVEN HILL BRANDS declined so dramatically. This could be due to several factors, such as:
    * **Product availability:** Are we facing supply chain issues or stockouts?
    * **Pricin